In [3]:
import re
import os
import json
import openai

model_engine = "gpt-3.5-turbo-0301"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
prompt = '''

Problema cliente, palavras-chave, soluções, comunicação clara, eficiência, sentimento final e um template dentro dos disponiveis.
Retorne somente um JSON no seguinte no formato:

{ 
"resume": "text",
"problem_identification": "text",
"sentiment_analysis": "negative" / "positive" / "mixed" / "neutral",
"relevant_topics": ["unique words with max 3"],
"solution_suggestion": ["short phrases with max 2"],
"template_response": ["Contatos de suporte" | "Problemas com financeiro" | "Problemas especificos" | "Não identificado"]
}

'''

In [5]:
def validate_json_format(json_data):
    """
    Validates if a JSON dictionary has the required keys and values.

    Parameters:
    json_data (dict): the JSON dictionary to be validated

    Returns:
    bool: True if the dictionary is validated or False if it is not.

    Example:
    >>> validate_json_format({"resume": "text", "problem_identification": "text", "sentiment_analysis": "negative", "relevant_topics": ["unique words with max 3"], "solution_suggestion": ["short phrases with max 2"]})
    True
    """
    required_keys = [
        "resume",
        "problem_identification",
        "sentiment_analysis",
        "relevant_topics",
        "solution_suggestion",
        "template_response"
    ]

    if not all(key in json_data for key in required_keys):
        return False

    if not isinstance(json_data["sentiment_analysis"], str):
        return False

    sentiments = {"negative", "positive", "mixed", "neutral"}
    if json_data["sentiment_analysis"] not in sentiments:
        return False

    if not isinstance(json_data["relevant_topics"], list):
        return False

    if not isinstance(json_data["solution_suggestion"], list):
        return False


    return True

In [6]:
def extract_json_from_text(text):
    """
    Extracts a JSON dictionary from a string using regular expressions.

    Parameters:
    text (str): the string containing the JSON dictionary

    Returns:
    dict: the extracted JSON dictionary

    Example:
    >>> extract_json_from_text("{ "resume": "text", "problem_identification": "text", "sentiment_analysis": "negative", "relevant_topics": ["unique words with max 3"], "solution_suggestion": ["short phrases with max 2"]}")
    {"resume": "text", "problem_identification": "text", "sentiment_analysis": "negative", "relevant_topics": ["unique words with max 3"], "solution_suggestion": ["short phrases with max 2"]}
    """
    
    pattern = re.compile(r'\{.*?\}')
    json_str = pattern.search(str(text).replace('\n', '')).group(0)
    dicionario_response = json.loads(json_str)
    return dicionario_response


In [7]:
def text_suporte(text):
    """
    Sends a request to the OpenAI Chat API and returns the response in JSON format.

    Parameters:
    text (str): the text message to be sent to the API

    Returns:
    dict: the JSON response from the API containing the resume, problem identification, sentiment analysis, relevant topics, and solution suggestion.

    Example:
    >>> chatgpt("Customer problem, keywords, solutions, clear communication, efficiency.")
    {"resume": "text", "problem_identification": "text", "sentiment_analysis": "negative" / "positive" / "mixed" / "neutral", "relevant_topics": ["unique words with max 3"], "solution_suggestion": ["short phrases with max 2"]}
    """
    success = False
    assistant_response = None
    contador = 0

    while not success:
        try:
            messages = [{"role": "system", "content": prompt}, {"role": "user", "content": text}]
            response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)
            assistant_response = dict(response["choices"][0])["message"]["content"]

            
            if contador >= 5:
                print("Sorry, but you have exceeded the limit of attempts... Reformulate the text and try again.")
                return { 
                        "resume": "Sorry, but you have exceeded the limit of attempts... Reformulate the text and try again. Could not extract from text: " + text,
                        "problem_identification": "Could not identify",
                        "sentiment_analysis": "neutral",
                        "relevant_topics": ["Could not identify"],
                        "solution_suggestion": ["Could not identify"],
                        "template_response":["Could not identify"]
                        }

            contador = contador + 1
            
            
            json_data = extract_json_from_text(assistant_response)

            if validate_json_format(json_data):
                success = True
            else:
                print("O JSON não está no formato esperado. Tentando novamente...")
                
            

        except Exception as e:
            print(f"Erro ao chamar a API: {e}")
            print("Tentando novamente...")

    return json_data

In [8]:
email = '''Assunto: Problemas com smartphone e insatisfação

Caro Atendente,

Recentemente, comprei um dos seus smartphones e, infelizmente, estou enfrentando alguns problemas com o aparelho. Ele apresenta dificuldades para carregar a bateria e a tela trava constantemente, tornando-o praticamente inutilizável. Estou bastante insatisfeito com a situação e gostaria de obter ajuda para resolver esses problemas.

Atenciosamente,

[Usuário]

---

Caro [Usuário],

Lamentamos saber que você está enfrentando problemas com o nosso produto e agradecemos por entrar em contato conosco. Vamos analisar o problema em duas partes. Primeiro, em relação ao carregamento da bateria, você poderia nos informar se está usando o carregador original que veio com o aparelho?

Em relação à tela travando, você já tentou reiniciar o aparelho ou restaurar as configurações de fábrica?

Aguardamos sua resposta.

Atenciosamente,

[Atendente]

---

Caro Atendente,

Sim, estou usando o carregador original e já tentei até outros carregadores compatíveis, mas o problema persiste. Já tentei reiniciar várias vezes, mas não adianta. Ainda não tentei restaurar as configurações de fábrica porque tenho medo de perder todos os meus dados.

Atenciosamente,

[Usuário]

---

Caro [Usuário],

Antes de restaurar as configurações de fábrica, recomendo que você faça um backup de todos os seus dados importantes, como fotos, vídeos e documentos. Depois disso, você pode tentar a restauração. Se o problema persistir, provavelmente será necessário encaminhar o aparelho para um centro de assistência técnica autorizado.

Quanto ao problema de carregamento da bateria, se ele persistir após a restauração das configurações de fábrica, acreditamos que o ideal seja levar o aparelho até uma assistência técnica autorizada. Eles poderão fazer uma análise mais detalhada do problema e solucioná-lo adequadamente.

Atenciosamente,

[Atendente]

---

Caro Atendente,

Fiz o backup e restaurei as configurações de fábrica conforme você sugeriu, e o problema da tela travando foi solucionado. Porém, o problema de carregamento da bateria ainda persiste. Agradeço a ajuda e o suporte que você me deu durante todo esse processo. Apesar da insatisfação inicial, estou satisfeito com o atendimento e a solução apresentada. Vou levar meu aparelho à assistência técnica e espero que tudo se resolva em breve.

Atenciosamente,

[Usuário]

---

Caro [Usuário],

Ficamos contentes em saber que o problema da tela foi resolvido e agradecemos por sua compreensão. Quanto ao problema de carregamento da bateria, esperamos que a assistência técnica autorizada possa solucioná-lo rapidamente.

Se você tiver alguma dúvida ou precisar de mais assistência, por favor, não hesite em entrar em contato conosco novamente. Agradecemos por escolher nossos produtos e desejamos que tudo se resolva em breve.

Atenciosamente,

[Atendente]'''

In [9]:
text_suporte(email)

{'resume': 'Usuário relata problemas com smartphone e insatisfação, mas recebe suporte do atendente',
 'problem_identification': 'Problemas com bateria do smartphone e tela travando',
 'sentiment_analysis': 'positive',
 'relevant_topics': ['smartphone',
  'suporte',
  'bateria',
  'tela travando',
  'assistência técnica'],
 'solution_suggestion': ['Restaurar configurações de fábrica',
  'Levar o aparelho à assistência técnica autorizada'],
 'template_response': 'Contatos de suporte'}

In [11]:
chat = '''
Usuário: Olá, estou enfrentando alguns problemas com o meu produto e estou bastante insatisfeito. Vocês poderiam me ajudar?

Atendente: Olá! Lamento saber que você está insatisfeito com o nosso produto. Estou aqui para ajudar. Por favor, me explique qual é o problema que você está enfrentando.

Usuário: Comprei um dos seus smartphones há apenas 2 meses, e ele já está apresentando problemas para carregar a bateria. Além disso, a tela trava constantemente, tornando-o praticamente inutilizável!

Atendente: Sinto muito por todo o transtorno que isso tem causado. Vamos analisar o problema em duas partes. Primeiro, em relação ao carregamento da bateria, você poderia me informar se está usando o carregador original que veio com o aparelho?

Usuário: Sim, estou usando o carregador original, e já tentei até outros carregadores compatíveis, mas o problema persiste.

Atendente: Entendi. Nesse caso, pode ser um problema no próprio aparelho ou na bateria. Vamos tentar solucionar o problema da tela travando. Você já tentou reiniciar o aparelho ou restaurar as configurações de fábrica?

Usuário: Já tentei reiniciar várias vezes, mas não adianta. Ainda não tentei restaurar as configurações de fábrica porque tenho medo de perder todos os meus dados.

Atendente: Compreendo sua preocupação. Antes de restaurar as configurações de fábrica, recomendo que você faça um backup de todos os seus dados importantes, como fotos, vídeos e documentos. Depois disso, você pode tentar a restauração. Se o problema persistir, provavelmente será necessário encaminhar o aparelho para um centro de assistência técnica autorizado.

Usuário: Ok, entendi. Vou tentar fazer o backup e restaurar as configurações de fábrica. Mas, e quanto ao problema de carregamento da bateria?

Atendente: Se o problema persistir após a restauração das configurações de fábrica, acredito que o ideal seja levar o aparelho até uma assistência técnica autorizada. Eles poderão fazer uma análise mais detalhada do problema e solucioná-lo adequadamente.

Usuário: Está bem, vou seguir suas instruções e depois entro em contato para informar se o problema foi resolvido.

*Algumas horas depois*

Usuário: Olá, fiz o backup e restaurei as configurações de fábrica conforme você sugeriu, e o problema da tela travando foi solucionado. Porém, o problema de carregamento da bateria ainda persiste.

Atendente: Fico feliz em saber que o problema da tela foi resolvido! Quanto ao problema de carregamento da bateria, recomendo que você leve o aparelho a uma assistência técnica autorizada, como mencionei anteriormente. Eles poderão identificar a causa do problema e solucioná-lo.

Usuário: Entendi. Agradeço a ajuda e o suporte que você me deu durante todo esse processo. Apesar da insatisfação inicial, estou satisfeito com o atendimento e a solução apresentada. Vou levar meu aparelho à assistência técnica e espero que tudo se resolva em breve.

Atendente: Fico contente em saber que pude ajudar e agradeço por sua compreensão. Se você tiver alguma dúvida ou precisar de mais assistência, por favor, não hesite em entrar em contato conosco novamente. Tenha um ótimo dia!
'''

In [12]:
text_suporte(chat)

{'resume': 'O cliente enfrentava problemas com um smartphone comprado há apenas dois meses. O aparelho apresentava problemas para carregar a bateria e uma tela que travava frequentemente. O atendente sugeriu algumas dicas para a resolução dos problemas, incluindo a checagem do carregador original e a restauração de configurações de fábrica. O problema da tela travada foi resolvido, mas o problema com a bateria persistiu. No final, o cliente ficou satisfeito com o atendimento e decidiu levar o aparelho a uma assistência técnica autorizada para solucionar o problema.',
 'problem_identification': 'Problemas de falha no carregamento da bateria e uma tela que trava constantemente em um smartphone comprado há apenas dois meses.',
 'sentiment_analysis': 'mixed',
 'relevant_topics': ['smartphone',
  'problemas tela',
  'problemas bateria',
  'carregador original',
  'assistência técnica'],
 'solution_suggestion': ['Faça um backup de todos os dados importantes e tente restaurar as configurações

In [14]:
email_zenvia = '''Assunto: Problemas com a portabilidade da API do WhatsApp e atendimento insatisfatório da Zenvia

Caro Atendente,

Recentemente, contratamos a empresa Zenvia para ser nossa plataforma de chatbot. Utilizamos por um tempo, e agora queremos migrar a nossa API oficial do WhatsApp (que hoje está com o Zenvia) para outra empresa. No entanto, estamos enfrentando problemas com a portabilidade, já que a Zenvia não realizou o processo corretamente e nosso atendimento via WhatsApp está parado há 2 semanas.

Além disso, estamos extremamente insatisfeitos com a falta de comunicação e o atendimento grosseiro por parte dos atendentes da Zenvia. Durante o processo de contratação, fomos bem atendidos, mas agora que queremos "descontratar", estamos enfrentando um verdadeiro pesadelo.

Gostaria de expressar nossa insatisfação e solicitar uma solução para que possamos resolver essa situação o mais rápido possível.

Atenciosamente,

[Felipe]

---

Caro Felipe,

Agradecemos por entrar em contato conosco. Lamentamos saber que você está enfrentando problemas com a portabilidade da API do WhatsApp e insatisfeito com o atendimento da Zenvia. Por favor, poderia informar a Razão Social e CNPJ/CPF cadastrados conosco? Caso tenha o código do ticket aberto com a solicitação de portabilidade, peço a gentileza de me informar.

Atenciosamente,

[Escalation Zenvia]

---

Caro Escalation Zenvia,

Agradeço a resposta. Seguem os dados solicitados:

Razão social: Autoserver Automação
CNPJ: *******1-20
TICKET: 00276348

Além disso, gostaria de informar que não localizamos nenhum e-mail enviado pela Zenvia recentemente.

Atenciosamente,

[Felipe]

---

Caro Felipe,

Obrigado por fornecer as informações. Conseguimos localizar o seu cadastro e acionamos o nosso time de Retention. Em breve, você receberá um retorno sobre a situação da portabilidade da API do WhatsApp e os próximos passos a serem seguidos.

Pedimos desculpas pelos inconvenientes causados e agradecemos sua paciência.

Atenciosamente,

[Escalation Zenvia]'''

In [15]:
text_suporte(email_zenvia)

{'resume': 'Problema com a portabilidade da API do WhatsApp e atendimento insatisfatório da Zenvia.',
 'problem_identification': 'O cliente contratou a Zenvia para ser sua plataforma de chatbot, porém está enfrentando problemas para migrar a API oficial do WhatsApp para outra empresa. A Zenvia não realizou o processo corretamente gerando a paralisação do atendimento via WhatsApp há duas semanas. Além disso, o atendimento da empresa tem sido insatisfatório.',
 'sentiment_analysis': 'negative',
 'relevant_topics': ['Zenvia',
  'atendimento insatisfatório',
  'migrar API',
  'portabilidade',
  'problemas',
  'WhatsApp'],
 'solution_suggestion': ['A Zenvia deveria priorizar a solução do problema de portabilidade para que o cliente possa migrar sua API para outra empresa. É importante que haja uma comunicação clara e suporte prestado pela Zenvia.',
  'Possível resolução de conflito via Mediação'],
 'template_response': 'Problemas com suporte'}

In [ ]:
chat_zenvia = '''Felipe: Olá, estou enfrentando problemas com a portabilidade da API do WhatsApp da Zenvia para outra empresa. Nosso atendimento via WhatsApp está parado há 2 semanas, e a equipe de suporte da Zenvia não está nos ajudando. Além disso, os atendentes têm sido grosseiros conosco. Gostaria de uma solução para esse problema.

Escalation Zenvia: Olá, Felipe! Lamentamos saber que você está enfrentando problemas com a portabilidade e insatisfeito com o atendimento da Zenvia. Por favor, poderia informar a Razão Social e CNPJ/CPF cadastrados conosco? Caso tenha o código do ticket aberto com a solicitação de portabilidade, peço a gentileza de me informar.

Felipe: Claro, aqui estão os dados:

Razão social: Autoserver Automação
CNPJ: *******1-20
TICKET: 00276348

E gostaria de informar que não recebemos nenhum e-mail recente da Zenvia.

Escalation Zenvia: Obrigado pelas informações, Felipe. Conseguimos localizar o seu cadastro e já acionamos o nosso time de Retention. Em breve, você receberá um retorno sobre a situação da portabilidade da API do WhatsApp e os próximos passos a serem seguidos. Pedimos desculpas pelos inconvenientes causados e agradecemos sua paciência.

Felipe: Agradeço a resposta e aguardo o contato do time de Retention. Espero que possamos resolver essa situação o mais rápido possível.

*Algumas horas depois*

Felipe: Olá, gostaria de informar que o time de Retention entrou em contato comigo e resolveu o problema da portabilidade da API do WhatsApp. Estou satisfeito com a rapidez e eficiência na resolução do problema. Agradeço o suporte e espero que o atendimento continue melhorando.

Escalation Zenvia: Olá, Felipe! Ficamos muito felizes em saber que o problema foi resolvido e que você está satisfeito com a solução apresentada. Agradecemos seu feedback e continuaremos trabalhando para melhorar nosso atendimento. Se precisar de mais assistência no futuro, por favor, não hesite em entrar em contato conosco. Tenha um ótimo dia!'''

In [16]:
text_suporte(chat_zenvia)

NameError: name 'chat_zenvia' is not defined